In [ ]:
%pip install langchain langchain_community openai python-dotenv duckduckgo-search  google-api-python-client 

In [3]:
from dotenv import load_dotenv
import os
load_dotenv() 
os.environ['GOOLE_API_KEY'][:5], os.environ['GOOGLE_CSE_ID'][:5]

('AIzaS', 'b3779')

RAG 테스트

In [27]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.llms import OpenAI
from langchain.tools import DuckDuckGoSearchResults
from googleapiclient.discovery import build

# OpenAI LLM 설정
llm = OpenAI(temperature=0.3)



def google_search(query, api_key, cse_id):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(
        q=query,
        cx=cse_id,
        lr='lang_ko',
        hl='ko',
        gl='kr'
    ).execute()
    items = res.get('items', [])
    results_text = []
    for item in items:
        title = item.get('title')
        snippet = item.get('snippet')
        link = item.get('link')
        results_text.append(f"제목: {title}\n요약: {snippet}\n링크: {link}\n")
    return "\n\n".join(results_text)

# Tool 등록 예
from langchain.agents import Tool

tools = [
    Tool(
        name="GoogleSearch",
        func=lambda q: google_search(q, os.environ['GOOLE_API_KEY'], os.environ['GOOGLE_CSE_ID']),
        description=(
            "이 도구는 입력된 한글 쿼리로 구글 한국어 검색을 수행합니다. "
            "검색 결과로 각 문서의 제목, 요약, 링크를 제공합니다. "
            "당신은 이 결과들을 종합해 질문에 대해 상세하고 구체적인 한글 답변을 만들어야 합니다. "
        )
    )
]

# RAG 에이전트 초기화
agent = initialize_agent(
    tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# 사용자의 질문에 대해 응답 처리
prompt_prefix = "아래 검색 결과를 참고하여 한국어로 상세하고 자연스럽게 답변해 주세요.\n"

user_input = "한국의 주요 이슈를 한국어로 자세히 알려줘."
try:
    response = agent.run(prompt_prefix + user_input)
except Exception as e:
    print(f'관련 자료를 찾을수 없습니다. : \n\n{e}')
else:
    print(response)




> Entering new AgentExecutor chain...
 I should use GoogleSearch to find relevant articles about major issues in Korea.
Action: GoogleSearch
Action Input: "한국의 주요 이슈"
Observation: 제목: [지급결제조사자료 2022-3] 암호자산 규제 관련 주요 이슈 및 입법 방향
요약: 한국은행 BANK OF KOREA · 한국은행 · 한국은행이 하는 일 · 설립목적 및 역사 · 한국은행법의 변천 ...
링크: https://www.bok.or.kr/portal/bbs/B0000232/view.do?nttId=10074134&menuNo=200706&pageIndex=1


제목: 멕시코 대선의 주요 이슈와 한국에 주는 시사점
요약: 2024. 5. 29. ... 본고에서는 현재 지지율 1, 2위를 기록 중인 셰인바움 후보와 갈베스 후보의 무역·통상 관련 주요 공약, 미국-멕시코. 간 무역·통상 이슈에 대한 두 후보 ...
링크: https://www.kiep.go.kr/galleryDownload.es?bid=0004&list_no=11314&seq=1


제목: 물적분할과 모자기업 동시상장의 주요 이슈 | | 보고서 | 자본시장연구원
요약: 2022. 6. 2. ... 최근 상장기업의 물적분할과 쪼개기상장이 사회적 이슈로 부상되었으나 대부분의 논의가 일부 대기업 사례에만 국한되어 있다.
링크: https://www.kcmi.re.kr/report/report_view?report_no=1489


제목: 멕시코 대선의 주요 이슈와 한국에 주는 시사점 | KIEP 세계경제 ...
요약: 2024. 5. 29. ... ▷ 미국-멕시코 간 주요 이슈로 △중국의 대(對)미국 우회 수출 증가, △멕시코의 에너지 정책, △USMCA 및 북미 공급망, △이민 등을 꼽을 수 있음.
링크: https://www.kiep.go.kr/gallery.